In [ ]:
import sys
import dataclasses

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, QWEN_MATH_GRPO_CONFIG, SMOL_LM_135M

cfg = QWEN_MATH_GRPO_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 1000
cfg.num_generations = 2
display(dataclasses.asdict(cfg))

In [ ]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()
wrapper.init_data_module()

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.train()